In [97]:
!pip install --upgrade numpy scipy scikit-learn

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/99/2b/f7114983d84303019385d93d24d729aedba67be7e083286f114188943cf3/numpy-1.26.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 222.6 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 220.2 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 292.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 298.1 kB/s eta 0:00:00
  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/43/d0/f3cd75b62e1b90f48dbf091261b2fc7ceec14a700e308c50f6a69c83d337/scipy-1.11.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\DELL\\anaconda3\\Lib\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Consider using the `--user` option or check the permissions.



In [108]:
!pip install --upgrade threadpoolctl

  Obtaining dependency information for threadpoolctl from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0


In [ ]:
!pip uninstall numpy
!pip install numpy==1.21.4

In [160]:
import pandas as pd
pd.set_option('display.max_rows', 200)
import numpy as np
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import math
from tqdm import tqdm
import os
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [3]:
def stem(text):
    try:
        ps = PorterStemmer()
        return ps.stem(text)
    except:
        print('error')
        return text

In [4]:
path_wiki="inputs/WikiWords_FirstMillion_Refined_V6.csv"

In [5]:
path_words="inputs/Google_sheet_2024.csv"

#### Wiki

In [6]:
wiki=pd.read_csv(path_wiki)
wiki.columns=['ENGLISH','OCCURENCES']
wiki=wiki[wiki['ENGLISH'].notna()]
wiki=wiki.drop_duplicates(subset=['ENGLISH'])
wiki['ENGLISH_stem']=wiki['ENGLISH'].apply(stem)
agg_dicte={'ENGLISH': 'first', 'OCCURENCES': 'sum'}
grouped_wiki = wiki.groupby('ENGLISH_stem').agg(agg_dicte).reset_index()
grouped_wiki=grouped_wiki[['ENGLISH_stem','OCCURENCES']]

#### Google Sheet's words

In [78]:
words=pd.read_csv(path_words)
words['ENGLISH_stem']=words['ENGLISH'].apply(stem)
words=words.drop_duplicates(subset=['ENGLISH'])
words=words.drop(columns='OCCURENCES')

#### Merge

In [79]:
words=words.merge(grouped_wiki,how='left',left_on=['ENGLISH_stem'],right_on=['ENGLISH_stem'])
words=words[words['OCCURENCES'].notna()]
words=words.sort_values(by='OCCURENCES', ascending=False)
words['ID']=words.index

#### Preprocess

In [80]:
langues=list(set(list(words.columns))-set(['ID','ENGLISH_stem','OCCURENCES']))
langues_bizarre=['CHINESE','KOREAN','JAPAN','RUSSIAN','ARABE','CHINESE']

In [122]:
words['INTERVAL']=pd.qcut(words['OCCURENCES'],100,duplicates='drop',labels=False)
words['INTERVAL']=100-words['INTERVAL']

In [48]:
def filter_latin(text):
    text_lower=text.lower()
    alphabet="abcdefghijklmnopqrstvwxyzéèêìîï"
    for letter in alphabet:
        if letter in text_lower:
            return ''
    return text
for langue in langues_bizarre:
    mask=words[langue].notna()
    words[langue]=words[mask][langue].apply(filter_latin)

In [55]:
def remove_punctuation(text):
    special_punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for ele in text:
        if ele in special_punc:
            text = text.replace(ele, " ")
    return text
for langue in langues:
    mask=words[langue].notna()
    words[langue]=words[mask][langue].apply(remove_punctuation)
    words[langue] = words[mask][langue].apply(lambda x: '' if x.strip() == '' else x)
    words[langue]=words[langue].fillna('')
    
    

In [137]:
import os
from fuzzywuzzy import fuzz
import warnings
warnings.filterwarnings('ignore')

In [175]:
for langue_speak in tqdm(langues):
    try:
        os.mkdir('output/dataset/'+langue_speak)
    except Exception as E:
        pass
    langues_learns=[langue for langue in langues if langue!=langue_speak]
    for langue_learn in langues_learns:

        df=words[['ID',langue_speak,langue_learn,'OCCURENCES']]
        
        df[langue_learn]=df[langue_learn].apply(str)
        df[langue_speak]=df[langue_speak].apply(str)
        
        df=df[(df[langue_learn].notna())&(df[langue_speak].notna())]
        
        df[langue_learn]=df[langue_learn].apply(lambda x: x.lower())
        df[langue_speak]=df[langue_speak].apply(lambda x: x.lower())
        
        df=df.drop_duplicates(subset=[langue_learn])
        df=df.drop_duplicates(subset=[langue_speak])
        
        df[langue_speak]=df[langue_speak].apply(lambda x: x.replace("'",' ').replace('"',' ').replace(',',' ').replace('{',' ').replace('}',' '))
        df[langue_learn]=df[langue_learn].apply(lambda x: x.replace("'",' ').replace('"',' ').replace(',',' ').replace('{',' ').replace('}',' '))
        
        df['_is_egal']=df[langue_speak]==df[langue_learn]
        df=df[~df['_is_egal']]
        
        df['_fuzzy_ratio'] = df.apply(lambda x: fuzz.ratio(x[langue_speak], x[langue_learn]), axis=1)
        df=df[df['_fuzzy_ratio']<70]
        df=df.drop(columns=['_is_egal','_fuzzy_ratio'])
        df['INTERVAL']=pd.qcut(df['OCCURENCES'],100,duplicates='drop',labels=False)
        df['INTERVAL']=100-df['INTERVAL']
        df['INTERVAL']=round((df['INTERVAL']+4)/104*100).apply(lambda x: int(x))
        df=df.drop(columns=['OCCURENCES'])
        df = df.sample(frac=1).reset_index(drop=True)
        
        df=df[(df[langue_learn].notna())&(df[langue_speak].notna())&(df[langue_learn]!='nan')&(df[langue_speak]!='nan')&(df[langue_learn]!='null')&(df[langue_speak]!='null')]
        #print(langue_speak+'_'+langue_learn+'.csv',len(df),df[langue_speak].isna().mean(),df[langue_learn].isna().mean())
        df.to_csv('output/dataset/'+langue_speak+'/'+langue_speak+'_'+langue_learn+'.csv',header=True,index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [01:29<00:00,  6.40s/it]


In [161]:
df

,ID,CHINESE,RUSSIAN,INTERVAL
13793,13793,到,к,5
13791,13791,这,а,5
13837,13837,你,ты,5
13794,13794,和,и,5
13839,13839,A,А,5
...,...,...,...,...
137,137,多孔,пористый,100
136,136,欧洲环保局,EPA,100
135,135,卡仕达酱,заварной крем,100
188,188,ROX.,рокс,100
